In [6]:
# External packages:
try:
    import cPickle as pickle  # cPickle is faster, but not as complete
    
except:
    import pickle

import math
import datetime
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

In [100]:
class CHWForecasting:
    
    def CHWForecasting(self, model_file = None):
        """Creates a chilled water forcasting object. Loads an earlier model from file if 'model_file' is specified."""
        self._hourly_model = None
        self._daily_model = None
        self._hrs_before = None
        self._hrs_after = None
        self._days_before = None
        self._days_after = None
        self._daily_significance = None
        
        if model_file is not None:
            # Unpickle here...
            self.load(model_file)
    
    def build_hourly_model(self, hourly_power, hourly_weather, hrs_before, hrs_after):
        """
        Build and train model using 'hourly power' and 'hourly weather' as training data.
        
        Use hrs_before and hrs_after to specify the range data should be windowed to. hrs_after should match the 
        numbers of hours the future preditions will be made on.
        """
        hourly = pd.merge(hourly_power, hourly_weather, on = 'DATE')
        hourly['DATE'] = hourly.apply(lambda x: pd.Timestamp(x['DATE']), axis=1)
        
        past_features = hourly.columns.tolist()
        past_features.remove('DATE')
        
        future_features = hourly_weather.columns.tolist()
        future_features.remove('DATE')

        #past_features = ['Total Power (trimmed)', 'HourlyDryBulbTemperature', 'HourlyWetBulbTemperature']
        #future_features = ['HourlyDryBulbTemperature', 'HourlyWetBulbTemperature']

        hourlyWindowed = self.windowData(hourly, npast = hrs_before, nfuture = hrs_after, colpast = past_features, colfuture = future_features)
        hourlyWindowed = self.genTSfeats(windowedDF = hourlyWindowed, windows = [22])  
        hourlyWindowed = self.addTimeFeatures(hourlyWindowed)
        X, Ys = self.genXY(hourlyWindowed, 'Total Power (trimmed)', maxoffset = hrs_after)
        
        all_model = []
        all_rmse = []
        
        for Y in Ys:
            model, mae, rmse, rsq = self.runModel(X, Y, model=ExtraTreesRegressor())
            all_model.append(model)
            all_rmse.append(rmse)
        
        self._hourly_model = [all_model, all_rmse]
        
        return self._hourly_model
    
    def build_daily_model(self, hourly_power, hourly_weather, days_before, days_after, significance = 0.85):
        """
        Build and train model using 'hourly power' and 'hourly weather' as training data.
        
        Use hrs_before and hrs_after to specify the range data should be windowed to. hrs_after should match the 
        numbers of hours in the future preditions will be made on.
        """
        train = pd.merge(hourly_power, hourly_weather, on = "DATE")
        self._daily_significance = significance
        self._daily_model = self.trainGB_ndays(train, significance = significance)
        
    def trainGB_ndays(self, df, predictorVar = 'Total Power (trimmed)_max', dropCols = ['DATE'], significance = 0.85):
        gb_modelList = []
        for i in range(5):
            days = i+1
            dailyMaxModels = self.trainGB_Model(df, days, predictorVar, dropCols, significance) 
            gb_modelList.append(dailyMaxModels)

        return gb_modelList
    
    def predict_hourly_forecast(self, date_and_time, recent_power, recent_weather, forecasted_weather):
        """ Creates a forecast object for this class using provided data to give predictions. """
        
        recent = pd.merge(recent_power, recent_weather, on = 'DATE')
        
        past_features = recent.columns.tolist()
        past_features.remove('DATE')
    
        future_features = forecasted_weather.columns.tolist()
        future_features.remove('DATE')
        
        merged = recent.append(forecasted_weather, sort = False)
        merged["DATE"] = merged.apply(lambda x: pd.Timestamp(x["DATE"]), axis=1)
        merged.reset_index()
        
        windowed = self.windowData(merged, npast = len(recent_power)-1, nfuture = len(forecasted_weather)-1, colpast = past_features, colfuture = future_features)
        windowed_ts = self.genTSfeats(windowedDF = windowed, windows = [22])
        self.addTimeFeatures(windowed_ts)
        
        df = windowed_ts.dropna()
        df = df.drop(columns = ['DATE'])
        
        all_pred = []

        for m in self._hourly_model[0]:
            all_pred.append(m.predict(df))
        
        max_power = max(all_pred) # peak power (tons)
        peak_future_hour = all_pred.index(max_power) 
        max_power_error = self._hourly_model[1][peak_future_hour] 
        pred_int = (max_power[0] - (2*max_power_error), max_power[0] + (2*max_power_error)) 
        
        current_date_and_time = pd.to_datetime(date_and_time) # assuming date_and_time gets passed in as a string in the form 'MM-DD-YYYY HH:MM:SS'
        hours_to_add = datetime.timedelta(hours = peak_future_hour + 1)
        peak_timestamp = current_date_and_time + hours_to_add
        
        return max_power, peak_timestamp, pred_int
    
    def predict_daily_forecast(self, recent_power, recent_weather, forecasted_weather, day_out=1):
        """Creates a forecast object for this class using provided data to give predictions."""
        predictDf = pd.merge(recent_power, recent_weather.append(forecasted_weather), on = 'DATE', how ='outer')
        return self.predict(predictDf, nDays=day_out, modelList=self._daily_model)
        
    def predict(self, df, nDays, modelList):
        test = self.prepPredict(df, nDays)
        date = test['DATE']
        testDf = pd.DataFrame(test.values.reshape(1,-1) , columns = test.index)

        X = testDf.drop(columns = ['DATE', 'Total Power (trimmed)_max'])

        lower = modelList[nDays - 1][0].predict(X)[0]
        mid = modelList[nDays - 1][1].predict(X)[0]
        upper = modelList[nDays - 1][2].predict(X)[0]

        print(f'{date}: {mid} ({lower},{upper})')
        return lower, mid, upper, date
    
    def windowData(self, df, npast=0, nfuture=0, colpast=[], colfuture=[]):
    
        # Add past windows:
        kept_past = df[colpast]  # elements used in past windows...
        for i in range(1, npast+1):
            temp = kept_past.shift(i)
            temp.columns = [f"{c}-{i}hr" for c in kept_past.columns]
            df = pd.concat([df,temp], axis=1)
        
        # Add future windows:
        kept_future = df[colfuture]  # elements used future windows...
        for i in range(1, nfuture+1):
            temp = kept_future.shift(-i)
            temp.columns = [f"{c}+{i}hr" for c in kept_future.columns]
            df = pd.concat([df,temp], axis=1)
    
        return df
    
    def genTSfeats(self, windowedDF, windows = []): 

        for win in windows: # Loop over windows of different sizes, if passed--22 is optimal.

            # Generate temporary 'lookback' dataframes for feature construction:
            tmp_power = windowedDF.iloc[:, 4:(3*win)+4:3]
            tmp_drybulb = windowedDF.iloc[:, 5:(3*win)+5:3]
            tmp_wetbulb = windowedDF.iloc[:, 6:(3*win)+6:3]

            tmp_dfs = [tmp_power, tmp_drybulb, tmp_wetbulb]
            var_names = ['power', 'drybulb', 'wetbulb']

            # Loop over temporary dataframes and construct desired features:
            for tmp, var in zip(tmp_dfs, var_names):
                
                # General statistics for base level.
                windowedDF[f'meanprev{win}_{var}'] = tmp.mean(axis=1)
                windowedDF[f'medianprev{win}_{var}'] = tmp.median(axis=1)
                windowedDF[f'minprev{win}_{var}'] = tmp.min(axis=1)
                windowedDF[f'maxprev{win}_{var}'] = tmp.max(axis=1)
                windowedDF[f'stdprev{win}_{var}'] = tmp.std(axis=1)

                # Capturing trend.
                windowedDF[f'mean_ewmprev{win}_{var}'] = tmp.T.ewm(com=9.5).mean().T.mean(axis=1)
                windowedDF[f'last_ewmprev{win}_{var}'] = tmp.T.ewm(com=9.5).mean().T.iloc[:,-1]
                windowedDF[f'avgdiff{win}_{var}'] = (tmp - tmp.shift(1, axis=1)).mean(axis=1)

        return windowedDF
    
    def addTimeFeatures(self, df):
        
        # Month:
        months = df.apply(lambda x: x["DATE"].month, axis=1)
        df['month_sin'] = np.sin((months-1)*(2.*np.pi/12))
        df['month_cos'] = np.cos((months-1)*(2.*np.pi/12))

        # Day of Week:
        dow = df.apply(lambda x: x["DATE"].dayofweek, axis=1)
        df['DoW_sin'] = np.sin(dow*(2.*np.pi/7))
        df['DoW_cos'] = np.cos(dow*(2.*np.pi/7))

        # Hour of Day:
        hr = df.apply(lambda x: x["DATE"].hour, axis=1)
        df['Hour_sin'] = np.sin(hr*(2.*np.pi/24))
        df['Hour_cos'] = np.cos(hr*(2.*np.pi/24))
        
        return df
    
    def addTimeFeatures_daily(self, df):
        months = df.apply(lambda x: x["DATE"].month, axis=1)
        df['month_sin'] = np.sin((months-1)*(2.*np.pi/12))
        df['month_cos'] = np.cos((months-1)*(2.*np.pi/12))

        df['DoW_sin'] = np.sin(df['weekday']*(2.*np.pi/7))
        df['DoW_cos'] = np.cos(df['weekday']*(2.*np.pi/7))
    
    def genXY(self, X, target_var, maxoffset = 1):
        '''
        Generate X and Y data to be used in run model. X should be untrimmed (include NaNs).
        Will return tuple contianing trimmed X and list of Y vectors with offset ranging from 1 to maxoffset.
        '''
        df = X.copy()
        Y = []

        for l in range(1, maxoffset+1):
            df[f"offset_{l}"] = df[target_var].shift(-l)

        df = df.dropna()
        df = df.reset_index(drop = True)
        df = df.drop(columns = ['DATE'])

        for l in range(1, maxoffset+1):
            Y.append(df[f"offset_{l}"])
            df.drop(columns=[f"offset_{l}"], inplace = True)

        return (df, Y)
    
    def runModel(self, X, Y, model, verbose=False):

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state = 12345, shuffle = True)  # Create train & test sets.
        model.fit(X_train, Y_train)  # Fit model...

        # Calculate error metrics:
        rsq = model.score(X_test, Y_test)
        rmse = math.sqrt(metrics.mean_squared_error(Y_test, model.predict(X_test)))
        mae = metrics.mean_absolute_error(Y_test, model.predict(X_test))

        # Display if selected in arguments:
        if verbose:
            print("R Squared Score: {:.4f}".format(rsq))
            print("Root Mean Squared Error: {:.2f}".format(rmse))
            print("Mean Absolute Error: {:.2f}".format(mae))

        return model, mae, rmse, rsq
    
    def formatTrainingData(self, df, trainDays = 7, nDays = 1):
    
        df["DATE"] = df.apply(lambda x: pd.Timestamp(x["DATE"]), axis=1)
        df['DATE'] = [d.date() for d in df["DATE"]]

        groupby = df.groupby('DATE').describe()

        colList = df.drop(columns = ['DATE']).columns
        parameters = ['min', 'max', 'mean']

        dailyData = {'DATE':list(groupby.index)}

        nday_cols = []
        for i in range(len(colList)):
            if(i == 0):
                dailyData['Total Power (trimmed)_max'] = list(groupby[colList[i]]['max'])
            else:                                     
                for param in parameters:
                    colname = colList[i] + "_" + param
                    dailyData[colList[i] + "_" + param] = list(groupby[colList[i]][param])
                    nday_cols.append(colname)

        daily = pd.DataFrame(dailyData)

        daily['weekday'] = [row['DATE'].weekday() for i, row in daily.iterrows()]
        if(nDays > 0):
            for col in nday_cols:
                for i in range(nDays + trainDays):
                    colname = col + "-" + str(i + 1) + "day"
                    daily[colname] = daily[col].shift(i+1)

            for col in ['Total Power (trimmed)_max']:#, 'Total Power (trimmed)_avg']:
                for i in range(nDays, trainDays + nDays + 1):
                    colname = col + "-" + str(i) + "day"
                    daily[colname] = daily[col].shift(i)

        self.addTimeFeatures_daily(daily)

        daily = daily.dropna()
        return daily
    
    def trainGB_Model(self, df, days, predictorVar, dropCols, significance):
        dropCols.append(predictorVar)

        daily = self.formatTrainingData(df, nDays = days)

        # Set lower and upper quantile
        LOWER_ALPHA = (1 - significance)/2
        UPPER_ALPHA = 1-LOWER_ALPHA

        # Each model has to be separate
        lower = GradientBoostingRegressor(loss="quantile",                   
                                                alpha=LOWER_ALPHA)
        # The mid model will use the default loss
        mid = GradientBoostingRegressor(loss="ls")

        upper = GradientBoostingRegressor(loss="quantile",
                                                alpha=UPPER_ALPHA)
        X = daily.drop(dropCols, axis=1)
        Y = daily[predictorVar]

        #train the 3 models separately
        lower.fit(X, Y)
        mid.fit(X, Y)
        upper.fit(X, Y)

        return [lower, mid, upper]
    
    def prepPredict(self, df, nDays, trainDays = 7):

        df["DATE"] = df.apply(lambda x: pd.Timestamp(x["DATE"]), axis=1)
        df['DATE'] = [d.date() for d in df["DATE"]]

        for index in range(len(df)):
            if(df.loc[index]['Total Power (trimmed)_max'] != df.loc[index]['Total Power (trimmed)_max']):
                today = index-1
                predictDay = today + nDays
                break
        #print(today)

        testDf = df.copy()

        if(nDays > 0):
            for col in testDf.columns[2:]:
                for i in range(nDays + trainDays):
                    colname = col + "-" + str(i + 1) + "day"
                    testDf[colname] = testDf[col].shift(i+1)

            powerCol = testDf.columns[1]
            for i in range(nDays, trainDays + nDays + 1):
                colname = powerCol + "-" + str(i) + "day"
                testDf[colname] = testDf[powerCol].shift(i)

        testDf['weekday'] = [row['DATE'].weekday() for i, row in testDf.iterrows()]
        self.addTimeFeatures_daily(testDf)

        return testDf.loc[predictDay]

    # load and save are untested, should work though
    # https://stackoverflow.com/questions/2709800/how-to-pickle-yourself
    
    def load(self, filename):
        """  """
        f = open(filename, 'rb')
        tmp_dict = pickle.load(f)
        f.close()          
        self.__dict__.clear()
        self.__dict__.update(tmp_dict) 

    def save(self, filename):
        """  """
        f = open(filename, 'wb')
        pickle.dump(self.__dict__, f, 2)
        f.close()

In [94]:
hourly_power = pd.read_csv('hourly_power.csv')
hourly_weather = pd.read_csv('hourly_weather.csv')

In [95]:
recent_power = pd.read_csv('recent_power.csv')
recent_weather = pd.read_csv('recent_weather.csv')
forecasted_weather = pd.read_csv('forecasted_weather.csv')

In [96]:
daily_forecasted_weather = pd.read_csv('daily_forecasted_weather.csv')
daily_recent_weather = pd.read_csv('daily_recent_weather.csv')
daily_recent_power = pd.read_csv('daily_recent_power.csv')

In [37]:
forecast = CHWForecasting()

In [13]:
hourly_model_w_errors = forecast.build_hourly_model(hourly_power, hourly_weather, 24, 24)

In [14]:
predictions = forecast.predict_hourly_forecast('2020-04-20 17:00:00', recent_power, recent_weather, forecasted_weather)

In [16]:
predictions

(array([3192.05078125]),
 Timestamp('2020-04-21 06:00:00'),
 (2292.833163584572, 4091.268398915428))

In [101]:
forecast = CHWForecasting()

In [102]:
forecast.build_daily_model(hourly_power, hourly_weather, days_before=7, days_after=5, significance = 0.85)

In [103]:
for i in range(5):
    forecast.predict_daily_forecast(daily_recent_power, daily_recent_weather, daily_forecasted_weather, day_out=i+1)

2019-06-30: 11275.641550280918 (12377.635676047397,16296.064352271864)
2019-07-01: 14272.834435484987 (11190.495730749943,15420.5163426715)
2019-07-02: 14397.382728538074 (9902.92163990516,17578.44544759516)
2019-07-03: 16867.580064366324 (12172.816733940328,17705.696337598405)
2019-07-04: 16983.863394559907 (12473.061727087459,18100.54754224029)


In [104]:
forecast.save("test")

In [109]:
load_forecast = CHWForecasting()

In [110]:
load_forecast.load("test")

In [111]:
for i in range(5):
    load_forecast.predict_daily_forecast(daily_recent_power, daily_recent_weather, daily_forecasted_weather, day_out=i+1)

2019-06-30: 11275.641550280918 (12377.635676047397,16296.064352271864)
2019-07-01: 14272.834435484987 (11190.495730749943,15420.5163426715)
2019-07-02: 14397.382728538074 (9902.92163990516,17578.44544759516)
2019-07-03: 16867.580064366324 (12172.816733940328,17705.696337598405)
2019-07-04: 16983.863394559907 (12473.061727087459,18100.54754224029)


In [63]:
daily_recent_power

,Unnamed: 0,DATE,Total Power (trimmed)_max
0,113,2019-06-22 23:00:00,9971.835401
1,114,2019-06-23 23:00:00,13077.625195
2,115,2019-06-24 23:00:00,13351.757068
3,116,2019-06-25 23:00:00,12941.737036
4,117,2019-06-26 23:00:00,13264.739941
5,118,2019-06-27 23:00:00,13099.856531
6,119,2019-06-28 23:00:00,12928.747217
7,120,2019-06-29 23:00:00,13139.119617


In [58]:
daily_recent_weather

,Unnamed: 0,DATE,HourlyWetBulbTemperature_max,HourlyDryBulbTemperature_max,HourlyWetBulbTemperature_min,HourlyDryBulbTemperature_min,HourlyWetBulbTemperature_mean,HourlyDryBulbTemperature_mean
0,113,2019-06-22 23:00:00,61.0,77.0,51.0,54.0,56.958333,66.666667
1,114,2019-06-23 23:00:00,64.0,81.0,52.0,54.0,58.625000,68.666667
2,115,2019-06-24 23:00:00,71.0,82.0,56.0,58.0,64.166667,71.791667
3,116,2019-06-25 23:00:00,70.0,79.0,61.0,66.0,66.041667,73.583333
4,117,2019-06-26 23:00:00,70.0,85.0,60.0,64.0,64.333333,74.625000
5,118,2019-06-27 23:00:00,68.0,86.0,58.0,60.0,64.458333,73.541667
6,119,2019-06-28 23:00:00,70.0,88.0,61.0,63.0,65.750000,76.250000
7,120,2019-06-29 23:00:00,71.0,83.0,63.0,68.0,67.916667,76.708333


In [61]:
daily_forecasted_weather

,Unnamed: 0,DATE,HourlyWetBulbTemperature_max,HourlyDryBulbTemperature_max,HourlyWetBulbTemperature_min,HourlyDryBulbTemperature_min,HourlyWetBulbTemperature_mean,HourlyDryBulbTemperature_mean
0,121,2019-06-30 23:00:00,63.0,74.0,56.0,59.0,60.291667,68.083333
1,122,2019-07-01 23:00:00,66.0,79.0,53.0,55.0,60.625000,69.666667
2,123,2019-07-02 23:00:00,71.0,80.0,60.0,67.0,66.875000,73.500000
3,124,2019-07-03 23:00:00,72.0,85.0,64.0,66.0,68.291667,76.250000
4,125,2019-07-04 23:00:00,75.0,89.0,63.0,65.0,70.791667,78.916667
